# FLOW_API Demonstration

## FLOW project meeting

### Leuven, 10-11 Sep 2024

In [1]:
import os
import xarray as xr
import matplotlib.pyplot as plt
from ncplot import view

from windIO.utils.yml_utils import validate_yaml
from windIO.utils import plant_schemas_path

from flow_api import run_foxes, run_pywake #, run_wayve, run_code_saturne

/home/jonas/venv/flow/lib/python3.12/site-packages/ncplot/plot.py:15: UserWarning: Unable to import cartopy. For better plots install cartopy or check cartopy installation!
  warnings.warn(


## Currently available example cases

In [2]:
!(cd ../examples/cases && tree -L 1)

.
├── AWAKEN
├── KUL_LES
├── open_source_scada
├── windio_4turbines
├── windio_4turbines_2flowcases
├── windio_4turbines_ABL
├── windio_4turbines_ABL_stable
└── windio_4turbines_profiles_stable

9 directories, 0 files


## Example 1: Four turbines in a row, homogeneous inflow timeseries

This case is called "windio_4turbines". The input data file structure looks like this:

In [3]:
!(cd ../examples/cases/windio_4turbines && tree)

.
├── plant_energy_resource
│   ├── FLOW_toy_study_energy_resource.yaml
│   └── Stochastic_atHubHeight.nc
├── plant_energy_site
│   └── FLOW_toy_study_energy_site.yaml
├── plant_energy_turbine
│   └── DTU_10MW_turbine.yaml
├── plant_wind_farm
│   └── FLOW_toy_study_wind_farm.yaml
└── wind_energy_system
    ├── FLOW_toy_study_analysis.yaml
    └── FLOW_toy_study_wind_energy_system.yaml

6 directories, 7 files


The main file of this case is called "FLOW_toy_study_wind_energy_system.yaml". It follows the windio schema. Notice how other files are referenced via the `!include` command:

In [4]:
!cat ../examples/cases/windio_4turbines/wind_energy_system/FLOW_toy_study_wind_energy_system.yaml

name: FLOW UQ vnv study on toy problem, 4 WT Wind Farm
site: !include ../plant_energy_site/FLOW_toy_study_energy_site.yaml
wind_farm: !include ../plant_wind_farm/FLOW_toy_study_wind_farm.yaml
attributes:
  flow_model:
    name: foxes
  analysis: !include FLOW_toy_study_analysis.yaml

  outputs:
    output_folder: "results"
    turbine_outputs:
        turbine_nc_filename: 'turbine_data.nc' # dimension = states, turbine
        output_variables: ['power', 'rotor_effective_velocity'] #'frequency'
    #
    flow_field:
      report: True
      flow_nc_filename: flow_field.nc
      cases_run:
        all_occurences: True
      output_variables: ['wind_speed', 'wind_direction']
      z_planes:
        z_sampling: "hub_height"
        xy_sampling: "default"
    #
    statistics:
      stats_filename: None
      AEP: False
      AEP_per_turbine: False
      power_percentiles:
        report: False
        percentiles: None


The `flow_model` under `attributes` states `foxes`, which means that a call of the main `flow_api` would ask `foxes` to compute results. However, this choice can be overruled by specified functions/commands.

**Provided python functions:**
- *run_api(input_yaml)*: Run the case with the flow model specified in the yaml file
- *run_foxes(input_yaml)*: Run the case with `foxes`
- *run_pywake(input_yaml)*: Run the case with `PyWake`
- *run_wayve(input_yaml)*: Run the case with `WAYVE`
- *run_code_saturne(input_yaml)*: Run the case with `code_saturne`

**Provided command-line tools:**
- *flow_api input_yaml*: Run the case with the flow model specified in the yaml file
- *flow_api_foxes input_yaml)*: Run the case with `foxes`
- *flow_api_pywake input_yaml)*: Run the case with `PyWake`
- *flow_api_wayve input_yaml)*: Run the case with `WAYVE`
- *flow_api_code_saturne input_yaml)*: Run the case with `code_saturne`

For example, we can run the main API call like as follows (Note that the "!" is only needed because we are in a notebook here, not a terminal):

In [5]:
!flow_api ../examples/cases/windio_4turbines/wind_energy_system/FLOW_toy_study_wind_energy_system.yaml

Validation succeeded
Reading windio file ../examples/cases/windio_4turbines/wind_energy_system/FLOW_toy_study_wind_energy_system.yaml
Ignoring 'z0', since no reference_height found. No ABL profile activated.
Running farm_calc
Selecting default engine 'MultiprocessEngine(chunk_size_states=None, chunk_size_points=None)'
Calculating 1000 states for 4 turbines
Computing 16 chunks using 16 processes
100%|██████████████████████████████████████████| 16/16 [00:00<00:00, 260.50it/s]
Running output: StateTurbineTable
Writing file results/turbine_data.nc
Running output: SliceData
Calculating data at 90000 points for 1000 states


Notice that the outputs that were defined in the `yaml` above triggered the writing of two output files:

In [6]:
!tree results

results
├── flow_field.nc
└── turbine_data.nc

1 directory, 2 files


In [7]:
from ncplot import view
view("results/flow_field.nc", vars= "WS")

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead


:DynamicMap   [state]
   :Image   [x,y]   (WS)